In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
            #print(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = "results/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    
    print('auc{0}, auprc{1}, acc{2}, F1{3} '.format(auc,auprc,acc,F1))
    
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01) nitin commented as its deprecated
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    opt = Adam(lr=0.001, decay = 0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model

In [3]:
# nitin added as without it the read_pickel below was giving the error as in notebook 5 when its writing
#pickel do a lazy execution and write the function as well
def mean(a):
    return sum(a) / len(a)

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

from collections.abc import Iterable, Iterator
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            print(type(embed),embed)
            print(isinstance(embed, Iterator))
            temp.append(embed)
            
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

tt = dict({ke:va})
create_dataset(tt)

for ke, va in temp_train_ner.items():
    print(ke, va)
    print(nit)

In [5]:
embedding_types = ['word2vec', 'fasttext']#, 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
target_problems = ['mort_hosp', 'mort_icu']#, 'los_3', 'los_7']


num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 11
unit_sizes = [128, 256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")
            
            temp_train_ner = dict((k, ner_word2vec[k]) for k in train_ids)
            temp_dev_ner = dict((k, ner_word2vec[k]) for k in dev_ids)
            temp_test_ner = dict((k, ner_word2vec[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min', period=1)


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    # nitin uncomment the code
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                               embed_name, each_problem, iteration, each_unit_size, 
                               each_layer, type_of_ner)
                    
                    #reset_keras(model) nitin commented as its not defined
                    #del model
                    sess = get_session()
                    clear_session()
                    sess.close()
                    #clear_session() nitin originally its only clear session but added abve three line
                    gc.collect()

Layer:  GRU
Hidden unit:  128
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
229/230 [============================>.] - ETA: 0s - loss: 0.3136 - acc: 0.8913
Epoch 00001: val_loss improved from inf to 0.25143, saving model to avg-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 7s 19ms/step - loss: 0.3135 - acc: 0.8914 - val_loss: 0.2514 - val_acc: 0.9124
Epoch 2/100
227/230 [============================>.] - ETA: 0s - loss: 0.2539 - acc: 0.9122
Epoch 00002: val_loss improved from 0.25143 to 0.24561, saving model to avg-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 4s 18ms/step - loss: 0.2549 - acc: 0.9120 - val_loss: 0.2456 - val_acc: 0.9153
Epoch 3/100
228/230 [============================>.] - ETA: 0s - loss: 0.2441 - acc: 0.9159
Epoch 00003: val_loss did not improve from 0.24561
230/230 [==============================] - 4s 18ms/step - loss: 0.2442 - acc: 0.9159 - val_loss

Epoch 9/100
229/230 [============================>.] - ETA: 0s - loss: 0.1582 - acc: 0.9480
Epoch 00009: val_loss did not improve from 0.17743
230/230 [==============================] - 6s 25ms/step - loss: 0.1582 - acc: 0.9480 - val_loss: 0.1780 - val_acc: 0.9416
Epoch 10/100
229/230 [============================>.] - ETA: 0s - loss: 0.1557 - acc: 0.9484
Epoch 00010: val_loss did not improve from 0.17743
230/230 [==============================] - 6s 26ms/step - loss: 0.1556 - acc: 0.9484 - val_loss: 0.1785 - val_acc: 0.9407
Epoch 11/100
228/230 [============================>.] - ETA: 0s - loss: 0.1552 - acc: 0.9492
Epoch 00011: val_loss did not improve from 0.17743
230/230 [==============================] - 6s 27ms/step - loss: 0.1553 - acc: 0.9493 - val_loss: 0.1783 - val_acc: 0.9411
Epoch 12/100
228/230 [============================>.] - ETA: 0s - loss: 0.1524 - acc: 0.9492
Epoch 00012: val_loss did not improve from 0.17743
230/230 [==============================] - 6s 26ms/step - l

Epoch 4/100
228/230 [============================>.] - ETA: 0s - loss: 0.1696 - acc: 0.9450
Epoch 00004: val_loss improved from 0.17926 to 0.17830, saving model to avg-word2vec-mort_icu-best_model.hdf5
230/230 [==============================] - 6s 26ms/step - loss: 0.1697 - acc: 0.9449 - val_loss: 0.1783 - val_acc: 0.9368
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.1653 - acc: 0.9472
Epoch 00005: val_loss improved from 0.17830 to 0.17674, saving model to avg-word2vec-mort_icu-best_model.hdf5
230/230 [==============================] - 6s 27ms/step - loss: 0.1652 - acc: 0.9472 - val_loss: 0.1767 - val_acc: 0.9402
Epoch 6/100
230/230 [==============================] - ETA: 0s - loss: 0.1607 - acc: 0.9467
Epoch 00006: val_loss did not improve from 0.17674
230/230 [==============================] - 6s 26ms/step - loss: 0.1607 - acc: 0.9467 - val_loss: 0.1771 - val_acc: 0.9383
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1583 - acc: 0.

Epoch 3/100
229/230 [============================>.] - ETA: 0s - loss: 0.1763 - acc: 0.9430
Epoch 00003: val_loss improved from 0.17756 to 0.17468, saving model to avg-word2vec-mort_icu-best_model.hdf5
230/230 [==============================] - 7s 29ms/step - loss: 0.1762 - acc: 0.9430 - val_loss: 0.1747 - val_acc: 0.9411
Epoch 4/100
228/230 [============================>.] - ETA: 0s - loss: 0.1705 - acc: 0.9450
Epoch 00004: val_loss improved from 0.17468 to 0.17385, saving model to avg-word2vec-mort_icu-best_model.hdf5
230/230 [==============================] - 7s 29ms/step - loss: 0.1707 - acc: 0.9448 - val_loss: 0.1738 - val_acc: 0.9416
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.1649 - acc: 0.9451
Epoch 00005: val_loss did not improve from 0.17385
230/230 [==============================] - 7s 29ms/step - loss: 0.1649 - acc: 0.9450 - val_loss: 0.1751 - val_acc: 0.9411
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.1633 - acc: 0.

Epoch 4/100
230/230 [==============================] - ETA: 0s - loss: 0.2341 - acc: 0.9175
Epoch 00004: val_loss improved from 0.24470 to 0.24349, saving model to avg-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 6s 26ms/step - loss: 0.2341 - acc: 0.9175 - val_loss: 0.2435 - val_acc: 0.9144
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.2283 - acc: 0.9191
Epoch 00005: val_loss improved from 0.24349 to 0.24230, saving model to avg-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 6s 26ms/step - loss: 0.2283 - acc: 0.9191 - val_loss: 0.2423 - val_acc: 0.9144
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.2225 - acc: 0.9207
Epoch 00006: val_loss improved from 0.24230 to 0.24139, saving model to avg-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 6s 26ms/step - loss: 0.2224 - acc: 0.9208 - val_loss: 0.2414 - val_acc: 0.9139
Epoch 7/100
230/230 [====

...

229/230 [============================>.] - ETA: 0s - loss: 0.1581 - acc: 0.9496
Epoch 00005: val_loss improved from 0.17665 to 0.17593, saving model to avg-fasttext-mort_icu-best_model.hdf5
230/230 [==============================] - 12s 51ms/step - loss: 0.1584 - acc: 0.9495 - val_loss: 0.1759 - val_acc: 0.9416
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.1531 - acc: 0.9502- ETA: 4s - loss: 0.1536
Epoch 00006: val_loss did not improve from 0.17593
230/230 [==============================] - 12s 51ms/step - loss: 0.1532 - acc: 0.9502 - val_loss: 0.1760 - val_acc: 0.9421
Epoch 7/100
230/230 [==============================] - ETA: 0s - loss: 0.1503 - acc: 0.9508
Epoch 00007: val_loss did not improve from 0.17593
230/230 [==============================] - 11s 49ms/step - loss: 0.1503 - acc: 0.9508 - val_loss: 0.1762 - val_acc: 0.9426
Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1483 - acc: 0.9513
Epoch 00008: val_loss improved from 0.1

Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1445 - acc: 0.9509
Epoch 00008: val_loss did not improve from 0.17332
230/230 [==============================] - 12s 53ms/step - loss: 0.1445 - acc: 0.9509 - val_loss: 0.1749 - val_acc: 0.9426
Epoch 9/100
229/230 [============================>.] - ETA: 0s - loss: 0.1425 - acc: 0.9519
Epoch 00009: val_loss did not improve from 0.17332
230/230 [==============================] - 12s 52ms/step - loss: 0.1424 - acc: 0.9519 - val_loss: 0.1748 - val_acc: 0.9421
Epoch 10/100
229/230 [============================>.] - ETA: 0s - loss: 0.1396 - acc: 0.9520- ETA: 1s - los
Epoch 00010: val_loss did not improve from 0.17332
230/230 [==============================] - 12s 52ms/step - loss: 0.1396 - acc: 0.9520 - val_loss: 0.1763 - val_acc: 0.9416
Epoch 11/100
229/230 [============================>.] - ETA: 0s - loss: 0.1395 - acc: 0.9517
Epoch 00011: val_loss did not improve from 0.17332
230/230 [==============================] -

Epoch 9/100
229/230 [============================>.] - ETA: 0s - loss: 0.1424 - acc: 0.9518
Epoch 00009: val_loss did not improve from 0.17378
230/230 [==============================] - 12s 54ms/step - loss: 0.1424 - acc: 0.9519 - val_loss: 0.1752 - val_acc: 0.9416
Epoch 10/100
229/230 [============================>.] - ETA: 0s - loss: 0.1388 - acc: 0.9535
Epoch 00010: val_loss did not improve from 0.17378
230/230 [==============================] - 12s 53ms/step - loss: 0.1387 - acc: 0.9535 - val_loss: 0.1759 - val_acc: 0.9416
Epoch 11/100
229/230 [============================>.] - ETA: 0s - loss: 0.1369 - acc: 0.9531
Epoch 00011: val_loss did not improve from 0.17378
230/230 [==============================] - 11s 50ms/step - loss: 0.1370 - acc: 0.9530 - val_loss: 0.1757 - val_acc: 0.9421
Epoch 12/100
230/230 [==============================] - ETA: 0s - loss: 0.1356 - acc: 0.9527- ETA: 5s - ETA: 2s - loss: 0.1359 - acc: 0.9 - ETA: 2s -
Epoch 00012: val_loss did not improve from 0.17378

Epoch 3/100
229/230 [============================>.] - ETA: 0s - loss: 0.2301 - acc: 0.9194
Epoch 00003: val_loss improved from 0.24100 to 0.23816, saving model to avg-fasttext-mort_hosp-best_model.hdf5
230/230 [==============================] - 12s 52ms/step - loss: 0.2300 - acc: 0.9193 - val_loss: 0.2382 - val_acc: 0.9172
Epoch 4/100
229/230 [============================>.] - ETA: 0s - loss: 0.2209 - acc: 0.9202
Epoch 00004: val_loss did not improve from 0.23816
230/230 [==============================] - 12s 53ms/step - loss: 0.2210 - acc: 0.9202 - val_loss: 0.2408 - val_acc: 0.9124
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.2144 - acc: 0.9219
Epoch 00005: val_loss did not improve from 0.23816
230/230 [==============================] - 13s 55ms/step - loss: 0.2143 - acc: 0.9219 - val_loss: 0.2436 - val_acc: 0.9120
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.2102 - acc: 0.9246- ETA: 0s - loss: 0.2103 - acc: 0.924
Epoch 00006: 

Epoch 10/100
229/230 [============================>.] - ETA: 0s - loss: 0.1993 - acc: 0.9290
Epoch 00010: val_loss did not improve from 0.24037
230/230 [==============================] - 14s 60ms/step - loss: 0.1992 - acc: 0.9291 - val_loss: 0.2421 - val_acc: 0.9124
Epoch 11/100
229/230 [============================>.] - ETA: 0s - loss: 0.1978 - acc: 0.9286
Epoch 00011: val_loss did not improve from 0.24037
230/230 [==============================] - 15s 65ms/step - loss: 0.1977 - acc: 0.9286 - val_loss: 0.2407 - val_acc: 0.9134
Epoch 12/100
229/230 [============================>.] - ETA: 0s - loss: 0.1957 - acc: 0.9294
Epoch 00012: val_loss did not improve from 0.24037
230/230 [==============================] - 13s 58ms/step - loss: 0.1961 - acc: 0.9294 - val_loss: 0.2441 - val_acc: 0.9134
auc0.8757330769257963, auprc0.5779211931411866, acc0.9160872692399904, F10.4712990936555891 
0.8757330769257963 0.5779211931411866 0.9160872692399904 0.4712990936555891
Problem type:  mort_icu
______